## basic test

In [1]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time


In [2]:
#-- REAL WORLD configuration, which is attached to the physics engine
# accessing this directly would be cheating!
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

In [15]:
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [16]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")

In [17]:
#-- using the viewer, you can view configurations or paths
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [83]:
#-- the following is the simulation loop
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    [y,J] = C.evalFeature(ry.FS.position, ["R_gripper"])
#     vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-1e-1];
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)
    

AttributeError: 'int' object has no attribute 'setJointState'

## doing things relative to an object

In [18]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
obj = C.addFrame("object")

In [19]:
# set frame parameters, associate a shape to the frame, 
# obj.setPosition([.1, 0, 1.4])
# obj.setQuaternion([np.sqrt(0.5), 0., 0., 0])
obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,.5,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])
V.setConfiguration(C)
V.recopyMeshes(C) #this is rarely necessary, only when you change meshes within C

# #Try adding a ball now
# ball = C.addFrame(name="ball")
# ball.setShape(ry.ST.sphere, [.1])
# ball.setPosition([.8,.8,1.5])
# ball.setColor([1,1,0])

# #Try adding a box now
# hand = C.addFrame(name="hand", parent="pr2L")
# obj.setShape(ry.ST.ssBox, size=[.2,.2,.1,.02]),
# obj.setPosition([.8,.8,1.5])
# obj.setColor([1,1,0])

In [20]:
#-- the following is the simulation loop
tau = .01

for t in range(900):
     time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    
    #some good old fashioned IK
    C.setJointState(q)                  #set your robot model to match the real q
    V.setConfiguration(C)               #to update your model display
    [y,J] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)
    

IndentationError: unindent does not match any outer indentation level (<ipython-input-20-aac80a8ca923>, line 8)

How could you align the gripper for a proper grasp?

In [21]:
S=0
V=0
C=0
RealWorld=0